In [63]:
from tifffile import imread
import os
from matplotlib import pyplot as plt
import json
import geojson
from scipy.io import loadmat
import h5py
import numpy as np

In [107]:
# pth_WSI_segmentations = r'\\10.99.68.178\andreex\data\Stardist\PDAC model\maybe_all_tiles_donald\test_WSI\StarDist_2_9_24\json'
pth_WSI_segmentations = r'\\10.99.68.178\andre\Eduarda PDAC\immune_cell_pipeline\star_dist_Donald_model\StarDist_1_29_23\json'

In [108]:
json_pth_list = sorted([os.path.join(pth_WSI_segmentations,file) for file in os.listdir(pth_WSI_segmentations) if file.endswith(".json")])
print(json_pth_list)

['\\\\10.99.68.178\\andre\\Eduarda PDAC\\immune_cell_pipeline\\star_dist_Donald_model\\StarDist_1_29_23\\json\\OM1R_618.json', '\\\\10.99.68.178\\andre\\Eduarda PDAC\\immune_cell_pipeline\\star_dist_Donald_model\\StarDist_1_29_23\\json\\OM2R_786.json', '\\\\10.99.68.178\\andre\\Eduarda PDAC\\immune_cell_pipeline\\star_dist_Donald_model\\StarDist_1_29_23\\json\\YM1R_210.json', '\\\\10.99.68.178\\andre\\Eduarda PDAC\\immune_cell_pipeline\\star_dist_Donald_model\\StarDist_1_29_23\\json\\YM2R_326.json']


In [109]:
ds = 4

In [121]:
def get_ds_data(segmentation_data, ds):
    data_list = []
    for data in segmentation_data:
        centroid = data['centroid'][0]
        contour = data['contour'][0]
    
        #print(centroid)
        #print(contour)
        
        ds_centroid = [int(c/ds) for c in centroid]
        ds_contour = [[value / ds for value in sublist] for sublist in contour]
        ds_contour = [[round(x,2), round(y,2)] for x, y in zip(ds_contour[0], ds_contour[1])]
        ds_contour = ds_contour[0:-1:4]  # make shape have 8 points instead of 32
        
        #print(ds_centroid)
        #print(ds_contour)
        
        dat = [ds_centroid, ds_contour]
        data_list.append(dat)
    return data_list

In [104]:
ds_contour

[[10934.80623626709, 11201.97412109375],
 [10934.602432250977, 11201.95980834961],
 [10934.29800415039, 11202.050605773926],
 [10933.905174255371, 11202.094825744629],
 [10933.480262756348, 11202.01545715332],
 [10933.125137329102, 11201.776596069336],
 [10933.14151763916, 11201.369674682617],
 [10933.32804107666, 11201.0],
 [10933.693473815918, 11200.740112304688],
 [10933.970420837402, 11200.573532104492],
 [10934.10383605957, 11200.40119934082],
 [10934.290817260742, 11200.290817260742],
 [10934.43189239502, 11200.149772644043],
 [10934.66340637207, 11200.187393188477],
 [10934.818840026855, 11200.089263916016],
 [10935.0, 11200.0228805542],
 [10935.202194213867, 11199.98348236084],
 [10935.404571533203, 11200.02326965332],
 [10935.685974121094, 11199.973358154297],
 [10935.871322631836, 11200.128677368164],
 [10936.040771484375, 11200.304580688477],
 [10935.984008789062, 11200.592407226562],
 [10936.033126831055, 11200.794494628906],
 [10936.010818481445, 11201.0],
 [10935.88898468

In [123]:
2/0.4416

4.528985507246377

In [124]:
# out_pth = r'\\10.99.68.178\andreex\data\Stardist\PDAC model\maybe_all_tiles_donald\test_WSI\StarDist_2_9_24\json\partial_jsons'

out_pth = r'\\10.99.68.178\andre\Eduarda PDAC\immune_cell_pipeline\star_dist_Donald_model\reduced_5x_geojsons'

# ds_amt = 4
ds_amt = 2/0.4416

for p, file in enumerate(json_pth_list):
    nm = file.split('\\')[-1]
    new_fn = os.path.join(out_pth, nm[:-5] + '.geojson')
    print(f'{p} / {len(json_pth_list)}')
    print(nm)
    
    if not os.path.exists(new_fn):
    
        segmentation_data = json.load(open(file))
        
        data_list = get_ds_data(segmentation_data, ds_amt)
    
        GEOdata = []
        
        for j, (centroid, contour) in enumerate(data_list):
            
            #if j == 100000:
            #    break
            
            centroid = [centroid[0] + 0, centroid[1] + 0]
            # xy coordinates are swapped, so I reverse them here with xy[::-1]
            # note: add 1 to coords to fix 0 indexing vs 1 index offset
            contour = [[coord+0 for coord in xy[::-1]] for xy in contour]  # Convert coordinates to integers
            contour.append(contour[0]) # stardist doesn't close the circle, needed for qupath
        
            # Create a new dictionary for each contour
            dict_data = {
                "type": "Feature",
                "id": "PathCellObject",
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [contour]
                },
                "properties": {
                    'objectType': 'annotation',
                    'classification': {'name': 'Nuclei', 'color': [97, 214, 59]}
                }
            }
        
            GEOdata.append(dict_data)
        
        with open(new_fn,'w') as outfile:
            geojson.dump(GEOdata,outfile)
        print('Finished',new_fn)
    
    else:
        print(f'skipping {new_fn}')

0 / 4
OM1R_618.json
Finished \\10.99.68.178\andre\Eduarda PDAC\immune_cell_pipeline\star_dist_Donald_model\reduced_5x_geojsons\OM1R_618.geojson
1 / 4
OM2R_786.json
Finished \\10.99.68.178\andre\Eduarda PDAC\immune_cell_pipeline\star_dist_Donald_model\reduced_5x_geojsons\OM2R_786.geojson
2 / 4
YM1R_210.json
Finished \\10.99.68.178\andre\Eduarda PDAC\immune_cell_pipeline\star_dist_Donald_model\reduced_5x_geojsons\YM1R_210.geojson
3 / 4
YM2R_326.json
Finished \\10.99.68.178\andre\Eduarda PDAC\immune_cell_pipeline\star_dist_Donald_model\reduced_5x_geojsons\YM2R_326.geojson
